### Sharpe Ratio Filter and Export Notebook

1. **Notebook Objective:**
	- This notebook calculates the Sharpe Ratio for a set of stocks obtained from company_tickers.json, which is approximately all NYSE stocks
    - It uses the Sharpe Ratio as a filter to exclude underperforming stocks; this filtering will be implemented in the next notebook to ensure a complete dataset is preserved for potential future use.	
    - Only stocks with a Sharpe Ratio above a certain threshold are retained
	- The filtered stock data is then exported for further analysis or reporting  

<br>

2. **Sharpe Ratio Formula:**
	- $\text{Sharpe Ratio} = \frac{\overline{R_p} - R_f}{\sigma_p}$
        - $\overline{R_p}$: Average return of the portfolio or stock
        - $R_f$: Risk-free rate
        - $\sigma_p$: Standard deviation of portfolio or stock returns


In [1]:
import time
import os 

start_time = time.time()

WORKSPACE_DIR = os.getenv('WORKSPACE_DIR')

if not os.getcwd().endswith('portfolio_py'):
    os.chdir(f'{WORKSPACE_DIR}/portfolio_py')
print(f'Current Working Directory: {os.getcwd()}')

from utils.finance_utils import calculate_sharpe_ratio
from utils.helpers import divide_chunks
from utils.config import PROGRAM_START_DATE, PROGRAM_END_DATE

from dotenv import load_dotenv

import yfinance as yf
import numpy as np
import pandas as pd

# from datetime import datetime
# from dateutil.relativedelta import relativedelta

Current Working Directory: /Users/blakeuribe/Desktop/portfolio_py

---------------------------------
finance_utils.py successfully loaded, updated last April. 29 2025 4:55
---------------------------------



---------------------------------
helpers.py successfully loaded, updated last Feb. 04 2025
---------------------------------


Updated on 04/15/2025 5:56


In [2]:
load_dotenv()

print(f'Ending Program at: {PROGRAM_END_DATE}')
print(f'Starting Program at: {PROGRAM_START_DATE}')

etf_df = pd.read_csv('data/raw/ETFs.csv')[['Symbol']] # include all etfs with NYSE stocks
etf_df.rename(columns={'Symbol': 'ticker'}, inplace=True)

nyse_ticker_df = pd.read_json(f'{WORKSPACE_DIR}/portfolio_py/data/raw/company_tickers.json')
nyse_ticker_df = nyse_ticker_df.T.drop(columns=['cik_str', 'title'])

ticker_df = pd.concat([etf_df, nyse_ticker_df]).drop_duplicates().reset_index(drop=True)


tbill_data = yf.download('^IRX', start=PROGRAM_START_DATE, end=PROGRAM_END_DATE, auto_adjust=True)['Close']
tbill_data = tbill_data / 100 / 360  # Convert to daily rate

spy_sharpe = calculate_sharpe_ratio(np.array('spy'), tbill=tbill_data, start_date=PROGRAM_START_DATE, end_date=PROGRAM_END_DATE)
print(f'Spy Sharpe: {spy_sharpe}')

Ending Program at: 2025-04-15
Starting Program at: 2024-04-15


[*********************100%***********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['^IRX']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')
INFO:backoff_logger:Starting call to 'utils.finance_utils.fetch_data_with_backoff', this is the 1st time calling it.
[*********************100%***********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['SPY']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


Spy Sharpe: Ticker
SPY   NaN
dtype: float64


In [3]:
ticker_df

,ticker
0,AAXJ
1,ACWI
2,ACWX
3,ADRA
4,ADRD
...,...
10984,EQV-WT
10985,EQV-UN
10986,CHARR
10987,CHARU


In [4]:
'AGA' in ticker_df['ticker'].to_list()

True

In [5]:

# Set chunk size and number of stocks
num_in_chunks = 15
num_of_stocks_to_get = 100 # Adjust as needed

# Get tickers and divide into chunks

tickers = ticker_df['ticker'][0:num_of_stocks_to_get]  # Remove .tolist() here
ticker_chunks = list(divide_chunks(tickers, num_in_chunks))

# print(f'Divided {num_of_stocks_to_get} tickers into {len(ticker_chunks)} chunks')

# Initialize an empty list to store results
df_list = []

# Loop through each chunk and process the stocks
for chunk in ticker_chunks:
    try:
        # Calculate Sharpe ratios for the current chunk
        sharpe_ratios_series = calculate_sharpe_ratio(
            chunk, 
            tbill=tbill_data, 
            start_date=PROGRAM_START_DATE, 
            end_date=PROGRAM_END_DATE
        )
        # Create a DataFrame for the chunk
        results_df_chunk = pd.DataFrame({
            'Tickers': sharpe_ratios_series.index, 
            'Sharpe_ratios': sharpe_ratios_series.values
        })
        df_list.append(results_df_chunk)

    except Exception as e:
        print(f'Error calculating Sharpe ratios for chunk: {chunk}\n{str(e)}')

# Concatenate all DataFrames vertically
final_df = pd.concat(df_list, axis=0, ignore_index=True)


# Ensure we have a benchmark value for futre use
final_df.loc[len(final_df)] = ['SPY', float(spy_sharpe.values[0])]

final_df = final_df.drop_duplicates()

# Check percentage of NaN values
pct_of_nan = (final_df.isna().sum().sum() / len(final_df) * 100).round(2)
print('\n----Df Report----')
print(f'Pct of NaN values is: {pct_of_nan}%')

# Save the results to CSV

if pct_of_nan >= 25:
    # Ask the user if they still want to export
    user_input = input(f"The percentage of NaN values is {pct_of_nan:.2f}%. Do you still want to export the DataFrame? (yes/no): ").strip().lower()

    if user_input == 'yes':
        final_df.to_csv(f'{WORKSPACE_DIR}/portfolio_py/data/clean/sharpe_ratios.csv', index=False)
        print('Df containing Sharpe Ratios Successfully Exported')
    else:
        print('Export cancelled.')
else:
    final_df.to_csv(f'{WORKSPACE_DIR}/portfolio_py/data/clean/sharpe_ratios.csv', index=False)
    print('Nan Values are below threshold, Successfully Exported')


end_time = time.time()
elapsed_time = end_time - start_time

print('\n----Time Report----')
print(f'Processing time: {elapsed_time:.2f} seconds, for {num_of_stocks_to_get} Tickers')

INFO:backoff_logger:Starting call to 'utils.finance_utils.fetch_data_with_backoff', this is the 1st time calling it.
[*********************100%***********************]  15 of 15 completed
ERROR:yfinance:
15 Failed downloads:
ERROR:yfinance:['ADRU', 'ACWI', 'ADRD', 'AGG', 'AGZ', 'AGQ', 'AAXJ', 'ADRA', 'ACWX', 'AGF', 'ADZ', 'AGA', 'AFK', 'AIA', 'ADRE']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')
INFO:backoff_logger:Starting call to 'utils.finance_utils.fetch_data_with_backoff', this is the 1st time calling it.
[*********************100%***********************]  15 of 15 completed
ERROR:yfinance:
15 Failed downloads:
ERROR:yfinance:['BDG', 'AOR', 'AOM', 'AYT', 'AOA', 'ALT', 'AOK', 'AXFN', 'BDD', 'AMJ', 'BAL', 'ASO', 'BBH', 'BABS', 'BAB']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')
INFO:backoff_logger:Starting call to 'utils.finance_utils.fetch_data_with_backoff', this is the 1st time calling it.
[*********************100%**********


----Df Report----
Pct of NaN values is: 100.0%
Export cancelled.

----Time Report----
Processing time: 17.81 seconds, for 100 Tickers


In [6]:
final_df.dropna()

,Tickers,Sharpe_ratios


In [7]:
import yfinance as yf
yf.download('msft')


YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['MSFT']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


Price,Adj Close,Close,High,Low,Open,Volume
Ticker,MSFT,MSFT,MSFT,MSFT,MSFT,MSFT
Date,,,,,,
